# Video Explanation

I creted this

[https://www.youtube.com/watch?v=MeY7KCvozOw](https://www.youtube.com/watch?v=MeY7KCvozOw)

In [17]:
import kagglehub

amazon = kagglehub.dataset_download("kritanjalijain/amazon-reviews")
google_vector = kagglehub.dataset_download("leadbest/googlenewsvectorsnegative300")

print(amazon)
print(google_vector)

/Users/tanishqsingh/.cache/kagglehub/datasets/kritanjalijain/amazon-reviews/versions/2
/Users/tanishqsingh/.cache/kagglehub/datasets/leadbest/googlenewsvectorsnegative300/versions/2


In [31]:
import tensorflow as tf
import pandas as pd

columns = ['sentiment', 'title', 'description']
df = pd.read_csv(f"{amazon}/train.csv", header=None, names=columns, chunksize=15_000)

In [20]:
import re
import string
import contractions
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tanishqsingh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tanishqsingh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
emoji_pattern = re.compile(
    "["
    u"\U0001F600-\U0001F64F"  # Emoticons
    u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # Transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # Flags
    u"\U00002700-\U000027BF"  # Dingbats
    u"\U000024C2-\U0001F251"  # Enclosed characters
    "]+", flags=re.UNICODE
)

def clean_text(df, column):
    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace(r"http\S+", "", regex=True)
    df[column] = df[column].str.replace(r"\d+", "", regex=True)
    df[column] = df[column].str.replace(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b", "", regex=True)
    df[column] = df[column].apply(lambda x: contractions.fix(x))
    df[column] = df[column].apply(lambda x: x.translate(str.maketrans("", "", string.punctuation)))
    df[column] = df[column].apply(lambda x: emoji_pattern.sub(r'', x))
    # df[column] = df[column].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))
    
    # df[column] = df[column].apply(lambda x: TextBlob(x).correct().string)
    df[column] = df[column].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))

    return df[column]

text_file = open('amazon.train.txt', 'w')

for idx, chunk in enumerate(df):
    print(idx)
    chunk.dropna(inplace=True)
    chunk.drop_duplicates(inplace=True)

    text_file.write(" ".join(clean_text(chunk, 'title').tolist()) + ' ')
    text_file.write(" ".join(clean_text(chunk, 'description').tolist()) + '\n')

text_file.close()

In [22]:
from gensim.models import Word2Vec
from gensim.models.word2vec import PathLineSentences
from gensim.models.callbacks import CallbackAny2Vec
import multiprocessing

In [ ]:
sentences = PathLineSentences("amazon.train.txt")
model = Word2Vec(vector_size=300, window=5, workers=multiprocessing.cpu_count() * 2, compute_loss=True, epochs=5)

In [119]:
model.build_vocab(sentences)

In [120]:
previous_loss = 0
for epoch in range(5):
    print(f"Epoch #{epoch + 1}")

    model.train(
        sentences,
        total_examples=model.corpus_count,
        epochs=1,
        compute_loss=True
    )
    
    current_loss = model.get_latest_training_loss()
    print(f"Loss: {current_loss - previous_loss}")
    previous_loss = current_loss

Epoch #1
Loss: 20881514.0
Epoch #2
Loss: -517128.0
Epoch #3
Loss: -236060.0
Epoch #4
Loss: 40044.0
Epoch #5
Loss: 352.0


In [ ]:
from gensim.models import KeyedVectors

w2v = KeyedVectors.load("amazon_vectors.h5")

In [32]:
import numpy as np

x_train_vectors = []
y_train_vectors = []

for idx, chunk in enumerate(df):
    if idx % 20 == 0:
        print(idx)

    chunk.dropna(inplace=True)
    chunk.drop_duplicates(inplace=True)

    chunk['title'] = clean_text(chunk, 'title')
    chunk['description'] = clean_text(chunk, 'description')

    x_train_vectors.append(chunk['description'].apply(lambda text: np.mean([w2v.wv[word] for word in text.split() if word in w2v.wv])).tolist())
    y_train_vectors.append(chunk['sentiment'] - 1)

    break

0


In [36]:
x_train = np.array(x_train_vectors)[0]
y_train = np.array(x_train_vectors)[0]

x_train.shape, y_train.shape

((14999,), (14999,))

In [39]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input

model = Sequential()

model.add(Input(shape=(1,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 128)            │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 385 (1.50 KB)

 Trainable params: 385 (1.50 KB)

 Non-trainable params: 0 (0.00 B)